In [70]:
import numpy as np
from torchvision.datasets import ImageFolder
from torchvision import transforms

In [71]:
# 导入训练图片的路径
trans_compose = transforms.Compose([
        #转换为torch数据类型
        transforms.ToTensor(),
        # transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5]),
        # 调整图片大小
        transforms.Resize((50,50))

        ])

ds = ImageFolder(r'C:\Users\seven\Downloads\imgs\raw-img',trans_compose)
ds 

Dataset ImageFolder
    Number of datapoints: 26179
    Root location: C:\Users\seven\Downloads\imgs\raw-img
    StandardTransform
Transform: Compose(
               ToTensor()
               Resize(size=(50, 50), interpolation=bilinear, max_size=None, antialias=True)
           )

In [72]:
ds.class_to_idx

{'cane': 0,
 'cavallo': 1,
 'elefante': 2,
 'farfalla': 3,
 'gallina': 4,
 'gatto': 5,
 'mucca': 6,
 'pecora': 7,
 'ragno': 8,
 'scoiattolo': 9}

In [73]:
ds.imgs

[('C:\\Users\\seven\\Downloads\\imgs\\raw-img\\cane\\OIF-e2bexWrojgtQnAPPcUfOWQ.jpeg',
  0),
 ('C:\\Users\\seven\\Downloads\\imgs\\raw-img\\cane\\OIP---A27bIBcUgX1qkbpZOPswHaFS.jpeg',
  0),
 ('C:\\Users\\seven\\Downloads\\imgs\\raw-img\\cane\\OIP---ZIdwfUcJeVxnh47zppcQHaFj.jpeg',
  0),
 ('C:\\Users\\seven\\Downloads\\imgs\\raw-img\\cane\\OIP---ZRsOF7zsMqhW30WeF8-AHaFj.jpeg',
  0),
 ('C:\\Users\\seven\\Downloads\\imgs\\raw-img\\cane\\OIP---_cJbI6Ei26w5bW1urHewHaCf.jpeg',
  0),
 ('C:\\Users\\seven\\Downloads\\imgs\\raw-img\\cane\\OIP---cByAiEbIxIAleGo9AqOQAAAA.jpeg',
  0),
 ('C:\\Users\\seven\\Downloads\\imgs\\raw-img\\cane\\OIP--0C2kq6I68beB3DduLoxMgHaEf.jpeg',
  0),
 ('C:\\Users\\seven\\Downloads\\imgs\\raw-img\\cane\\OIP--0CZnfLuBERVtzRm2QbB3gHaK4.jpeg',
  0),
 ('C:\\Users\\seven\\Downloads\\imgs\\raw-img\\cane\\OIP--1QXriWyOTJg-9fEwbznmgHaI4.jpeg',
  0),
 ('C:\\Users\\seven\\Downloads\\imgs\\raw-img\\cane\\OIP--2uTtzjaszBfEMCROuLN3wHaLH.jpeg',
  0),
 ('C:\\Users\\seven\\Downloads\\im

In [74]:
import torch

from torch.utils.data import Dataset, DataLoader, random_split
from torch.utils.data import DataLoader
# 划分训练和测试的数据
train, test = random_split(ds, (25000, 1179) )
# 设定每次读取的数量
data_loader = torch.utils.data.DataLoader(dataset=train,batch_size=200, shuffle=True)
data_loader2 = torch.utils.data.DataLoader(dataset=test,batch_size=200, shuffle=True)


In [75]:
import torch.nn as nn

class MyNet(nn.Module):
    def __init__(self):
        super(MyNet, self).__init__()
        self.L1 = nn.Linear(7500,100)
        self.L2 = nn.Linear(100,200)
        self.L = nn.Linear(200,10)

    def forward(self, x):
        #降二维，从start_dim开始相乘
        x = torch.flatten(x,start_dim = 1)
        g1 = self.L1(x)
        # f1=torch.sigmoid(g1)
        f1=torch.relu(g1)

        g2 = self.L2(f1)
        f2 = torch.relu(g2)

        g = self.L(f2)

        return g
    
model = MyNet()

In [76]:
import torchmetrics

lossf = nn.CrossEntropyLoss()     #loss函数
optimizer = torch.optim.SGD(model.parameters(), lr=0.03)  #SGD函数
metricsf = torchmetrics.Accuracy(task='multiclass', num_classes=10)       #torchmetrics函数

for i in range(25):
    for batchX, batchY in data_loader:
        score = model(batchX)
        score = torch.squeeze(score)
        loss = lossf(score, batchY)

        loss.backward() #求导
        optimizer.step() #梯度下降
        optimizer.zero_grad() #梯度清零

        mse = metricsf(score, batchY)    
    print(loss, metricsf.compute())
    metricsf.reset()

tensor(2.1420, grad_fn=<NllLossBackward0>) tensor(0.2031)
tensor(2.0573, grad_fn=<NllLossBackward0>) tensor(0.2488)
tensor(1.9840, grad_fn=<NllLossBackward0>) tensor(0.2745)
tensor(1.9425, grad_fn=<NllLossBackward0>) tensor(0.2896)
tensor(1.7795, grad_fn=<NllLossBackward0>) tensor(0.3086)
tensor(1.9636, grad_fn=<NllLossBackward0>) tensor(0.3210)
tensor(1.8267, grad_fn=<NllLossBackward0>) tensor(0.3282)
tensor(1.7716, grad_fn=<NllLossBackward0>) tensor(0.3407)
tensor(1.7318, grad_fn=<NllLossBackward0>) tensor(0.3469)
tensor(1.8976, grad_fn=<NllLossBackward0>) tensor(0.3564)
tensor(1.7726, grad_fn=<NllLossBackward0>) tensor(0.3624)
tensor(1.7363, grad_fn=<NllLossBackward0>) tensor(0.3666)
tensor(1.8787, grad_fn=<NllLossBackward0>) tensor(0.3723)
tensor(1.7485, grad_fn=<NllLossBackward0>) tensor(0.3786)
tensor(1.6678, grad_fn=<NllLossBackward0>) tensor(0.3862)
tensor(1.7658, grad_fn=<NllLossBackward0>) tensor(0.3904)
tensor(1.8070, grad_fn=<NllLossBackward0>) tensor(0.3954)
tensor(1.6892,

In [ ]:
torch.save(model, 'model.pth')

In [77]:
for batchX, batchY in data_loader2:
    score = model(batchX)
    score = torch.squeeze(score)
    mse = metricsf(score, batchY)  
      
print(metricsf.compute())

tensor(0.3851)
